# V 1.2

### A versão 1.2 tem o formato completo do cromagrama sem a primeira e a ultima oitava e usa a detecção de acordes da music21

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from music21 import *


In [9]:
def cria_chroma_from_sample(sample, sample_rate):

    y_harmonic = librosa.effects.harmonic(sample)

    # Compute chroma features from the harmonic signal
    # Transforma cada faixa de frequencia em notas musicais
    #chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
     #                                       sr=sample_rate)
     
    chromagram = librosa.amplitude_to_db(np.abs(librosa.cqt(y_harmonic, sr=sample_rate)), ref=np.max)
    
    chromagram = chromagram[12:-12]

    return chromagram

In [10]:
def identifica_picos(lista):

    n1, n2, n3 = 0, 1, 2
    picos_notas = []

    #and lista[n2] > -130
    # verifica de 3 em 3, procurando picos
    while n3 < len(lista) :

        # testa se há a condição de pico e se atende a quantidade mínima de decibeis
        if lista[n1] < lista[n2] > lista[n3] and lista[n2] > -1900:
            picos_notas.append([n2, lista[n2]])

        n1, n2, n3 = n1+1, n2+1, n3+1

    # ordena com base no volume
    picos_notas = sorted(picos_notas, key=lambda x: x[1], reverse=True)

    return picos_notas

In [20]:
def identifica_fundamental(chroma):

    #critério de seleção: linha com o maior valor
    linhas_somadas = np.sum(chroma, axis=1)

    # argsort() retorna os índices que ordenariam um array.
    # [-2:] pega os dois últimos índices, que correspondem às maiores somas
    #i_maiores_somas = np.argsort(linhas_somadas)[-15:]
    
    picos_notas = identifica_picos(linhas_somadas)

    notas_musicais = {
    0: "C",
    1: "C#",
    2: "D",
    3: "D#",
    4: "E",
    5: "F",
    6: "F#",
    7: "G",
    8: "G#",
    9: "A",
    10: "A#",
    11: "B"
    }

    array_notas = []

    for i in picos_notas:
        nota = notas_musicais[i[0]%12]
        
        array_notas.append(nota)

    
    if array_notas:
        acorde = chord.Chord(array_notas)
        nota_fundamental = str(acorde.root())
    else:
        nota_fundamental = '0'

    #print(array_notas)
    return nota_fundamental
    

In [12]:
def contar_sequencias_notas(lista):
    # recebe uma sequencia de notas e transforma em uma matriz 2x2 com a nota e a quantidade de tempos que ela se repetiu
    
    contagens_sequenciais = []
    atual = lista[0]
    contagem = 0

    for elemento in lista:
        if elemento == atual:
            contagem += 1
        else:
            contagens_sequenciais.append([atual, contagem])
            atual = elemento
            contagem = 1
    contagens_sequenciais.append([atual, contagem])  # Para o último elemento

    # Sua lista de entrada
    #entrada = [1, 1, 1, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 2, 2]
    #saida   = [[1, 3], [2, 2], [3, 4], [1, 4], [2, 2]]

    # Chamar a função para contar sequências
    #resultado = contar_sequencias_notas(entrada)

    return contagens_sequenciais

In [13]:
def main(file = str):
    # Carregar o arquivo mp3
    audio = AudioSegment.from_file(file, format="mp3")

    # Definindo a duração de cada segmento em milissegundos
    #duracao_segmento = 167  # 1/6 segundos
    duracao_segmento = 500  # 1/6 segundos

    notas_fundamentais_acordes = []

    # Segmentando o áudio
    for i in range(0, len(audio), duracao_segmento):
        segmento = audio[i:i + duracao_segmento]
        #print(i, i+duracao_segmento)

        sample = np.array(segmento.get_array_of_samples()).astype(np.float32) / 32768
        sample_rate = segmento.frame_rate

        chromagram = cria_chroma_from_sample(sample=sample, sample_rate=sample_rate)
        
        fundamental = identifica_fundamental(chroma=chromagram)

        notas_fundamentais_acordes.append(fundamental)

    
    notas_fundamentais_acordes = contar_sequencias_notas(notas_fundamentais_acordes)
    return(notas_fundamentais_acordes)

In [21]:
notas_final = main('teste2_1min.mp3')

as notas estão indo entre 7 e 7 unidades de tempo escolhidas. Com variação de + - 1

Bug das quintas foi quase completamente corrigido

In [18]:
notas_final

[['0', 3],
 ['G', 1],
 ['0', 2],
 ['A', 5],
 ['F', 5],
 ['C', 5],
 ['C#', 1],
 ['G', 5],
 ['A', 3],
 ['D#', 1],
 ['A', 1],
 ['F', 6],
 ['C', 5],
 ['G', 5],
 ['A', 5],
 ['F#', 1],
 ['F', 5],
 ['C', 4],
 ['A', 1],
 ['G', 2],
 ['E', 2],
 ['G', 1],
 ['C', 1],
 ['A', 4],
 ['C', 1],
 ['F', 5],
 ['C', 5],
 ['G', 6],
 ['A', 5],
 ['F', 5],
 ['C', 6],
 ['G', 2],
 ['E', 2],
 ['G', 1],
 ['A', 4],
 ['F#', 2],
 ['F', 2]]